#### imports and wrappers

In [2]:
root_dir = r'/home/yukim/Desktop/prescription'
#root_dir = r'C:\\Users\\yukim\\github\\prescription'

from warnings import warn
import Levenshtein as L
import os
import pandas as pd
import pickle
import re

os.chdir(root_dir)

#windows環境ではsymspellをimportできないので,少しややこい処理
try:
    import vision1_basics as v1_b
except ModuleNotFoundError:
    print('vision1_basics was not imported')

#本サーバーではこの関数内で読み込むのではなく, どこかに既に読み込まれているDBを参照する.
with open('df_medicineInfo.pickle','rb') as f:
    df_medicine_info = pickle.load(f)

loading hospital symspell...


In [3]:
class wrapper_class(object):
    '''本処理実施に必要なvision1_basics.pyにある関数を格納したabstract class
    windows環境ではsymspellをload出来ないため必要'''
    
    def __init__(self):
        pass
    
    def convert_days(self,str_,genngou):
        '''XX年XX月XX日の形のstringを受取、genngouにて表された元号番号を添えて
        JAHIS規格で返す処理'''

        year = str_.split('年')[0]
        month = str_.split('年')[1].split('月')[0]
        day = str_.split('月')[1].split('日')[0]

        l = [re.sub(' ','',year),re.sub(' ','',month),re.sub(' ','',day)]

        for i in range(len(l)):
            if len(l[i])==1:
                l[i] = '0'+l[i]

        return str(genngou) + str(l[0]) + str(l[1]) + str(l[2])

    def symbol_num2(self,text):  #123・456みたいなのを123と456に分ける
        text=re.sub('[^0-9.,・-]', '', text)
        split_search = re.search('[.,・-]', text)
        if split_search is not None:
            id_23_insurance_card_id=text.split(split_search.group())[0]
            id_23_insurance_card_num=text.split(split_search.group())[1]
            return id_23_insurance_card_id, id_23_insurance_card_num
        else:
            return "??","??"

import sys
if 'vision1_basics' not in sys.modules:
    print('wrapper of vision1_b in use')
    v1_b = wrapper_class()

#### 入力データの可視化/定義

In [4]:
#data vizualize

data_num,csv_dir=148,os.path.join(root_dir,'csv_data/')
csv_base = pd.read_csv(csv_dir+str(data_num)+'_base.csv',encoding='shift-jis').fillna('None')
csv_meds = pd.read_csv(csv_dir+str(data_num)+'_medi.csv',encoding='shift-jis').fillna('None')

csv_meds

,RP_num,RP_ren,name,youryou,youhou,tyouzai,zaikei_kubun,unit
0,1,1,【般】 アゾセミド錠30mg,2.0,1日1回朝食後,14,1,錠
1,2,1,【般】スピロノラクトン錠25mg,1.0,1日1回朝食後,14,1,錠
2,3,1,ワーファリン錠1mg,1.5,1日1回朝食後,14,1,錠
3,4,1,ラニラピッド錠0.05mg,1.0,1日1回朝食後,14,1,錠
4,5,1,【般】リマプロストアルファデクス錠5μg,1.0,1日1回朝食後,14,1,錠
5,6,1,【般】アンブロキソール塩酸塩徐放口腔内崩壊錠45mg,1.0,1日1回朝食後,14,1,錠
6,7,1,ウルティブロ吸入用カプセル,14.0,1日1回朝,1,3,カプセル
7,8,1,ロキソニンテープ100mg 10cm×14cm,35.0,両肩・胸部・両下肢部へ1日3枚,1,3,枚
8,9,1,【般】ロキソプロフェンNa錠60mg,1.0,疼痛時,14,1,錠
9,10,1,【般】チザニジン錠1mg,1.0,1日1回夕食後,14,1,錠


In [5]:
input_list_init = ['06271310', '1234・5678', '患者次郎', 'カンジャジロウ', '昭和', '10年12月7日', '女', '被保険者', '平成', '10年12月7日',
          'あｓｄｆ', 'ダミー', 'ダミー', 'ダミー', 'あｓｄｆ',
          'RpNum,薬品1,ペルジピン散10%,1,-1,g,カプセル', 
          'RpNum,薬品2,コセンティクス皮下注150mgペン 1mL,1,-1,キット,カプセル',
          'RpNum,薬品3,アルト原末 1g,1,-1,管,カプセル,how-to-use,ダミー,用法未取得,-1,how-to-use,ダミー,用法未取得,-1,how-to-use,ダミー,用法未取得,-1,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー,ダミー',
          '']

In [6]:
input_list_PHC = ["06271340", "1234・5678", "患者次郎", "カンジャジロウ", "昭和", "44年9月6日", "男", "被保険者", "平成", "26年12月1日", 
              "田中クリニック", "1", "0411306", "12", "区分保険者扶養者...医師氏名dummy?", 
              "RpNum,薬品1,カルボシステイン錠250mg,1,-1,ダミー,カプセル", 
              "RpNum,薬品2,アンブロキソール塩酸塩錠15mg,1,3,錠,カプセル", 
              "RpNum,薬品3,フスコデ配合錠,1,-1,錠,カプセル", 
              "RpNum,薬品4,セフカペンピボキシル塩酸塩錠100mg,1,-1,ダミー,カプセル,how-to-use,ダミー,用法未取得,-1,how-to-use,ダミー,用法未取得,-1,how-to-use,ダミー,用法未取得,-1,how-to-use,内服,分3毎食後服用,14", 
              "RpNum,薬品5,ニフェジピン徐放錠20mg(24時間持続),2,2,錠,カプセル,how-to-use,内服,分2朝・夕食後服用,28", 
              "RpNum,薬品6,ケトプロフェンテープ40mg(10×14cm非温感),3,-1,ダミー,カプセル,how-to-use,外用,1日1回貼付,-1",
              ""]

In [7]:
input_list_PHC_with_error = ["", "1234・5678", "", "", "昭和", "44年9月6日", "男", "被保険者", "平成", "26年12月1日", 
              "", "1", "0411306", "12", "区分保険者扶養者...医師氏名dummy?", 
              "RpNum,薬品1,カルボシステイン錠250mg,1,-1,ダミー,カプセル", 
              "RpNum,薬品2,アンブロキソール塩酸塩錠15mg,1,3,錠,カプセル", 
              "RpNum,薬品3,フスコデ配合錠,1,-1,錠,カプセル", 
              "RpNum,薬品4,セフカペンピボキシル塩酸塩錠100mg,1,-1,ダミー,カプセル,how-to-use,ダミー,用法未取得,-1,how-to-use,ダミー,用法未取得,-1,how-to-use,ダミー,用法未取得,-1,how-to-use,内服,分3毎食後服用,14", 
              "RpNum,薬品5,ニフェジピン徐放錠20mg(24時間持続),2,2,錠,カプセル,how-to-use,内服,分2朝・夕食後服用,28", 
              "RpNum,薬品6,ケトプロフェンテープ40mg(10×14cm非温感),3,-1,ダミー,カプセル,how-to-use,外用,1日1回貼付,-1",
              ""]

#### 情報が欠損している際のダミー値params定義

In [8]:
#dummy値の定義

dict_dummy_basics = {}

dict_dummy_basics['1_医療機関レコード'] = {}
dict_dummy_basics['1_医療機関レコード']['医療機関コード種別'] = '-1'
dict_dummy_basics['1_医療機関レコード']['医療機関コード']  = '-1'
dict_dummy_basics['1_医療機関レコード']['医療機関都道府県コード']  = '-1'
dict_dummy_basics['1_医療機関レコード']['医療機関名称'] = 'テスト医療機関'

dict_dummy_basics['5_医師レコード'] = {}
dict_dummy_basics['5_医師レコード']['医師漢字氏名'] = 'テスト医師'

dict_dummy_basics['11_患者氏名レコード'] = {}
dict_dummy_basics['11_患者氏名レコード']['患者漢字氏名'] = 'テスト患者'
dict_dummy_basics['11_患者氏名レコード']['患者カナ氏名'] = 'テストカンジャ'
    
dict_dummy_basics['12_患者性別レコード'] = {}
dict_dummy_basics['12_患者性別レコード']['患者性別'] = '-1'

dict_dummy_basics['13_患者生年月日レコード'] = {}
dict_dummy_basics['13_患者生年月日レコード']['患者生年月日'] = '-1'

dict_dummy_basics['22_保険者番号レコード'] = {}
dict_dummy_basics['22_保険者番号レコード']['保険者番号'] = '-1'
    
dict_dummy_basics['23_記号番号レコード'] = {}
dict_dummy_basics['23_記号番号レコード']['被保険者証記号'] = '-1'
dict_dummy_basics['23_記号番号レコード']['被保険者証番号'] = '-1'
dict_dummy_basics['23_記号番号レコード']['被保険者/被扶養者'] = '-1'

dict_dummy_basics['51_処方箋交付年月日レコード'] = {}
dict_dummy_basics['51_処方箋交付年月日レコード']['処方箋交付年月日'] = '-1'


dict_dummy_meds = {}
l_keys = ['RP番号','RP連番','剤形区分','調剤数量','用法名称','薬品名称','用量','単位名']

l_vals = ['-1','-1','-1','-1','ダミー用法','ダミー医薬品','-1','ダミー単位']

dict_dummy_meds = dict(zip(l_keys,l_vals))

In [9]:
#入力情報が欠損であると判定する文字列の定義
l_deficit_vals = ['','用法未取得','-1','ダミー']

#### function definition

In [10]:
#データ成型処理群

dict_gengou_ids = {'明治':1, '大正':2, '昭和':3, '平成':4, '令和':5}
dict_patient_sex_ids = {'男':'1','女':'2'}
dict_insurance_type_ids = {'被保険者':'1','被扶養者':'2'}

def search_df_med(med_name,l_names=df_medicine_info['name'].values.tolist(),df=df_medicine_info):
    '''医薬品データベースを検索する処理'''
    
    cnt = l_names.count(med_name)
    if cnt==0:
        
        changed_med_name,min_dist = 'dummy',100000
        l=[]
        for medicine in l_names:
            dist = L.distance(med_name,medicine)
            l.append(dist)
            if dist<min_dist:
                changed_med_name = medicine
                min_dist = dist
        
        warn(med_name+'was not in DB and was changed to '+changed_med_name)
        med_name =changed_med_name            
        
    elif cnt>=2:
        
        warn(med_name+' has more than one id')
        
    _, _, _, zaikei_kubun, _2_rp_code, _, _, _, _7_ippan_code, _ = df[df['name']==med_name].iloc[0].values.tolist()
    
    if _2_rp_code is not None:
        code_type = '2'
        code = _2_rp_code
    elif _7_ippan_code is not None:
        code_type = '7'
        code = _7_ippan_code
    else:
        raise ValueError('something is wrong.')
        
    return med_name,zaikei_kubun,code_type,code

def append(high_key,low_keys,low_vals,dict_basic_info,l_dificit_vals=l_deficit_vals):
    '''患者基本情報を構成する上位レコード名称string, 下位レコード名称list, レコード値listを入力とし,
    それが欠損値であるか否かを判定/代入した上で, 患者基本情報dictにその情報を付与する処理'''

    #欠損情報は''で表されていると仮定し, Noneでfillする
    for i in range(len(low_vals)):
        if low_vals[i] in l_deficit_vals:
            low_vals[i] = None

    dict_basic_info[high_key] = dict(zip(low_keys,low_vals))
    
    return True

def unpack_csv_to_dicts(data_num,csv_dir=os.path.join(root_dir,'csv_data/')):
    '''正解データcsvのdirを受け取り, それをJAHIS文字列化可能なdictの形に整形する処理
    RP番号end以外の用法は"用法未取得"として入力されると想定'''
        
    dict_basic_info, dict_med_info = {},{}
    csv_base = pd.read_csv(csv_dir+str(data_num)+'_base.csv',encoding='shift-jis').fillna('None')
    csv_meds = pd.read_csv(csv_dir+str(data_num)+'_medi.csv').fillna('None')
        
    #basic_infoの格納
    l_contents_basic = csv_base['content'].values.tolist()
    
    def append(high_key,low_keys,low_vals,dict_basic_info=dict_basic_info):
        
        #欠損情報は''で表されていると仮定し, Noneでfillする
        for i in range(len(low_vals)):
            if low_vals[i] == '':
                low_vals[i] = None
        
        dict_basic_info[high_key] = dict(zip(low_keys,low_vals))
               
        
    ##1_医療機関レコードunpacking
    append('1_医療機関レコード',
           ['医療機関コード種別','医療機関コード','医療機関都道府県コード','医療機関名称'],
           l_contents_basic[0:4],dict_basic_info)
    
    ##5_医師レコードunpacking
    append('5_医師レコード',
          ['医師漢字氏名'],
          [l_contents_basic[4]],dict_basic_info)

    ##11_患者氏名レコードunpacking
    append('11_患者氏名レコード',
           ['患者漢字氏名','患者カナ氏名'],
           l_contents_basic[5:7],dict_basic_info)
    
    ##12_患者性別レコードunpacking
    append('12_患者性別レコード',
           ['患者性別'],
          [l_contents_basic[7]],dict_basic_info)
    
    ##13_患者生年月日レコードunpacking
    append('13_患者生年月日レコード',
           ['患者生年月日'],
           [l_contents_basic[8]],dict_basic_info)
     
    ##22_保険者番号レコードunpacking
    append('22_保険者番号レコード',
           ['保険者番号'],
           [l_contents_basic[9]],dict_basic_info)
    
    ##23_記号番号レコード
    append('23_記号番号レコード',
           ['被保険者証記号','被保険者証番号','被保険者/被扶養者'],
           l_contents_basic[10:13],dict_basic_info)
    
    ##51_処方箋交付年月日レコード
    append('51_処方箋交付年月日レコード',
           ['処方箋交付年月日'],
           [l_contents_basic[13]],dict_basic_info)        
    
    
    #med_infoの格納
    l_keys = ['RP番号','RP連番','剤形区分','調剤数量','用法名称','薬品名称','用量','単位名']
    for i in range(len(csv_meds)):
        
        RP_num,RP_ren_num,med_name,youryou_num,youhou,chouzai_num,zaikei_kubun,unit = csv_meds.loc[i].values.tolist()
        if zaikei_kubun == 'None':
            med_name,zaikei_kubun,_,_ = search_df_med(med_name)
            
        RP_num_nxt = RP_num+1 if i==len(csv_meds)-1 else csv_meds.at[i+1,'RP_num']
        if RP_num == RP_num_nxt:
            zaikei_kubun,chouzai_num,youhou = ['not RP end']*3
                        
        l_vals = [str(RP_num), str(RP_ren_num), zaikei_kubun, str(chouzai_num), youhou, med_name, str(youryou_num), unit]
        l_vals = [None if i in l_deficit_vals else i for i in l_vals]
        dict_med_info[i] = dict(zip(l_keys,l_vals))

    return dict_basic_info, dict_med_info

def unpack_list_to_dicts(input_list):
    '''修正画面UIよりの結果をlistとして受け取り, それをJAHIS文字列化可能なdictの形に整形する処理'''
    #本来はstring形式で受け取っているので, そっちに変更する可能性アリ
    #用法が存在する医薬品の位置が少し不明瞭であり, 実運用時はここを要確認
    
    dict_basic_info, dict_med_info = {},{}
    
    #basic_infoの格納    
        
    ##1_医療機関レコードunpacking
    append('1_医療機関レコード',
           ['医療機関コード種別','医療機関コード','医療機関都道府県コード','医療機関名称'],
           [input_list[11],input_list[12],input_list[13],input_list[10]],dict_basic_info)
    
    ##5_医師レコードunpacking
    append('5_医師レコード',
          ['医師漢字氏名'],
          [input_list[14]],dict_basic_info)

    ##11_患者氏名レコードunpacking
    append('11_患者氏名レコード',
           ['患者漢字氏名','患者カナ氏名'],
           input_list[2:4],dict_basic_info)
    
    ##12_患者性別レコードunpacking
    append('12_患者性別レコード',
           ['患者性別'],
          [dict_patient_sex_ids[input_list[6]]],dict_basic_info)
    
    ##13_患者生年月日レコードunpacking
    birthday,birthday_gengou = input_list[5],dict_gengou_ids[input_list[4]]
    append('13_患者生年月日レコード',
           ['患者生年月日'],
           [str(v1_b.convert_days(birthday,birthday_gengou))],dict_basic_info)
     
    ##22_保険者番号レコードunpacking
    append('22_保険者番号レコード',
           ['保険者番号'],
           [input_list[0]],dict_basic_info)
    
    ##23_記号番号レコード
    id_,num = v1_b.symbol_num2(input_list[1])
    append('23_記号番号レコード',
           ['被保険者証記号','被保険者証番号','被保険者/被扶養者'],
           [id_,num,dict_insurance_type_ids[input_list[7]]],dict_basic_info)
    
    ##51_処方箋交付年月日レコード
    append('51_処方箋交付年月日レコード',
           ['処方箋交付年月日'],
           [str(v1_b.convert_days(input_list[9],dict_gengou_ids[input_list[8]]))],dict_basic_info)
            
    #med_infoの格納
    l_keys = ['RP番号','RP連番','剤形区分','調剤数量','用法名称','薬品名称','用量','単位名']
    RP_num,RP_ren_num,update_RP_num = 1,0,False

    cnt = 0
    for str_med in input_list[15:-1]:
        
        list_med = str_med.split(',')
        
        zaikei_kubun,chouzai_num,youhou = ['not RP end']*3
        med_name,youryou_num,youryou_unit = list_med[2],list_med[4],list_med[5]
        
        if update_RP_num:
            RP_num += 1
            RP_ren_num = 1
            update_RP_num = False
        else:
            RP_ren_num+=1
        
        if 'how-to-use' in list_med:
            zaikei_kubun,chouzai_num,youhou = list_med[-3:]
            update_RP_num = True
                
        l_vals = [str(RP_num),str(RP_ren_num),zaikei_kubun,str(chouzai_num),youhou,med_name,str(youryou_num),youryou_unit]
        l_vals = [None if i in l_deficit_vals else i for i in l_vals]
        
        dict_med_info[cnt] = dict(zip(l_keys,l_vals))
        cnt+=1
    
    return dict_basic_info , dict_med_info

In [11]:
def look_up_basic_dict(dict_basic_info,key1,key2,dict_dummy_basics=dict_dummy_basics):
    '''指定されたkeyの患者基本情報をlookupし, それが欠損情報の場合はダミー値を代入する処理'''
    
    val = dict_basic_info[key1][key2]
    if val is None:
        val = dict_dummy_basics[key1][key2]
    return val

def look_up_med_dict(dict_med_info,key1,key2,dict_dummy_meds=dict_dummy_meds):
    '''指定されたkeyの医薬品情報をlookupし, それが欠損情報の場合はダミー値を代入する処理'''
    
    val = dict_med_info[key1][key2]
    if val is None:
        val = dict_dummy_meds[key2]    
    return val

def create_basic_str(dict_basic_info,JAHIS_ver,dummy_vals='abstract_input_now',JAHIS_ver_algo='JAHIS7'):
    '''情報が整備された患者基本情報dictを受け取り, それをJAHIS規格文字列に変換して返す処理'''
    
    assert JAHIS_ver==JAHIS_ver_algo, 'JAHIS ver must match'
    
    _1_hospital_code_type=look_up_basic_dict(dict_basic_info,'1_医療機関レコード','医療機関コード種別')
    _1_hospital_code=look_up_basic_dict(dict_basic_info,'1_医療機関レコード','医療機関コード')
    _1_hospital_place_code=look_up_basic_dict(dict_basic_info,'1_医療機関レコード','医療機関都道府県コード')
    id_1_hospital_name=look_up_basic_dict(dict_basic_info,'1_医療機関レコード','医療機関名称')
    
    _5_doctor_name_kanji = look_up_basic_dict(dict_basic_info,'5_医師レコード','医師漢字氏名')
    
    id_11_patient_name_kanji = look_up_basic_dict(dict_basic_info,'11_患者氏名レコード','患者漢字氏名')
    id_11_patient_name_kana = look_up_basic_dict(dict_basic_info,'11_患者氏名レコード','患者カナ氏名')
    
    id_12_patient_sex = look_up_basic_dict(dict_basic_info,'12_患者性別レコード','患者性別')
    
    _13_patient_birthday = look_up_basic_dict(dict_basic_info,'13_患者生年月日レコード','患者生年月日')
     
    id_22_insurance_patient_num = look_up_basic_dict(dict_basic_info,'22_保険者番号レコード','保険者番号',)
       
    id_23_insurance_card_id = look_up_basic_dict(dict_basic_info,'23_記号番号レコード','被保険者証記号') 
    id_23_insurance_card_num = look_up_basic_dict(dict_basic_info,'23_記号番号レコード','被保険者証番号')
    _23_insurance_type = look_up_basic_dict(dict_basic_info,'23_記号番号レコード','被保険者/被扶養者')

    _51_prescription_date = look_up_basic_dict(dict_basic_info,'51_処方箋交付年月日レコード','処方箋交付年月日')
    
    basic_result = (
        "1"+","+_1_hospital_code_type + "," + _1_hospital_code + "," + _1_hospital_place_code + "," + id_1_hospital_name + "\n"
        +"5"+","+ "" + "," + "" + "," + _5_doctor_name_kanji + "\n"
        +"11"+","+ "" + "," + id_11_patient_name_kanji + "," + id_11_patient_name_kana + "\n"
        +"12"+","+ id_12_patient_sex + "\n" 
        +"13"+","+ _13_patient_birthday + "\n"
        +"22"+","+id_22_insurance_patient_num+"\n"
        +"23"+","+id_23_insurance_card_id+","+id_23_insurance_card_num+","+_23_insurance_type+","+""+"\n"
        +"51"+","+_51_prescription_date+"\n")
    
    return basic_result

def create_med_str(dict_med_info,JAHIS_ver,dummy_vals='abstract_input_now',JAHIS_ver_algo='JAHIS7'):
    '''情報が整備された処方医薬品情報dictを受け取り, それをJAHIS規格文字列に変換して返す処理
    RPの終わりではない医薬品は, 剤形区分&用法&調剤数量が'not RP end'で渡されていると想定'''
    #281_薬品補足レコードは未対応
    
    assert JAHIS_ver==JAHIS_ver_algo, 'JAHIS ver must match'
    
    med_result = ''
    
    l_keys = list(dict_med_info.keys())
    l_keys.sort()
    
    l_201s = []
    for key in l_keys:
        
        RP_num,RP_ren_num = look_up_med_dict(dict_med_info,key,'RP番号'),look_up_med_dict(dict_med_info,key,'RP連番')
        zaikei_kubun = look_up_med_dict(dict_med_info,key,'剤形区分')
        chouzai_num = look_up_med_dict(dict_med_info,key,'調剤数量')
        youhou = look_up_med_dict(dict_med_info,key,'用法名称')
        
        med_name = look_up_med_dict(dict_med_info,key,'薬品名称')
        med_name, _, med_code_type,med_code = search_df_med(med_name)
        youryou_num,youryou_unit = look_up_med_dict(dict_med_info,key,'用量'),look_up_med_dict(dict_med_info,key,'単位名')
        
        str_201 = (
            "201"+","+RP_num+","+RP_ren_num+",,"
            +med_code_type+","+med_code+","+med_name+","+youryou_num+","+"1"+","+youryou_unit+"\n")
            
        if youhou != 'not RP end':
            med_result += "101"+","+RP_num+","+zaikei_kubun+","+""+","+chouzai_num+"\n"
            med_result += "111"+","+RP_num+","+"1"+","+""+","+youhou+","+"\n"
            for str_ in l_201s:
                med_result+=str_
            med_result+=str_201
            l_201s = []
        
        else:
            l_201s.append(str_201)

        RP_num_prev = RP_num
        
    return med_result

#### MAIN

In [15]:
#MAIN処理
def create_JAHIS_str(input_,input_type,JAHIS_ver='JAHIS7'):
    
    if input_type==list:
        dict_basic_info,dict_med_info = unpack_list_to_dicts(input_)
    elif input_type=='csv_num':
        dict_basic_info,dict_med_info = unpack_csv_to_dicts(input_)
    else:
        raise ValueError(input_type,'is unsupported.')
        
    print(dict_med_info)
        
    JAHIS_str = JAHIS_ver + '\n' + create_basic_str(dict_basic_info,JAHIS_ver) + create_med_str(dict_med_info,JAHIS_ver)
        
    return JAHIS_str

#### 実行結果samples

In [16]:
print(create_JAHIS_str(input_list_PHC,input_type=list))

{0: {'RP番号': '1', 'RP連番': '1', '剤形区分': 'not RP end', '調剤数量': 'not RP end', '用法名称': 'not RP end', '薬品名称': 'カルボシステイン錠250mg', '用量': None, '単位名': None}, 1: {'RP番号': '1', 'RP連番': '2', '剤形区分': 'not RP end', '調剤数量': 'not RP end', '用法名称': 'not RP end', '薬品名称': 'アンブロキソール塩酸塩錠15mg', '用量': '3', '単位名': '錠'}, 2: {'RP番号': '1', 'RP連番': '3', '剤形区分': 'not RP end', '調剤数量': 'not RP end', '用法名称': 'not RP end', '薬品名称': 'フスコデ配合錠', '用量': None, '単位名': '錠'}, 3: {'RP番号': '1', 'RP連番': '4', '剤形区分': '内服', '調剤数量': '分3毎食後服用', '用法名称': '14', '薬品名称': 'セフカペンピボキシル塩酸塩錠100mg', '用量': None, '単位名': None}, 4: {'RP番号': '2', 'RP連番': '1', '剤形区分': '内服', '調剤数量': '分2朝・夕食後服用', '用法名称': '28', '薬品名称': 'ニフェジピン徐放錠20mg(24時間持続)', '用量': '2', '単位名': '錠'}, 5: {'RP番号': '3', 'RP連番': '1', '剤形区分': '外用', '調剤数量': '1日1回貼付', '用法名称': None, '薬品名称': 'ケトプロフェンテープ40mg(10×14cm非温感)', '用量': None, '単位名': None}}
JAHIS7
1,1,0411306,12,田中クリニック
5,,,区分保険者扶養者...医師氏名dummy?
11,,患者次郎,カンジャジロウ
12,1
13,3440906
22,06271340
23,1234,5678,1,
51,4261201
101,1,内服,,分3毎食後服用
111,1,1

In [14]:
print(create_JAHIS_str(1011,input_type='csv_num'))

JAHIS7
1,1,4224481,13,仙川駅前すずきクリニック
5,,,鈴木　次郎
11,,松田　悠希,ﾏﾂﾀﾞ ﾕｳｷ
12,1
13,4070306
22,138271
23,２７ー１８,9796,1,
51,5020729
101,1,1,,5
111,1,1,,1日2回 朝・夕食後,
201,1,1,,7,4413004F1ZZZ,メキタジン錠3mg,2,1,錠
101,2,1,,5
111,2,1,,1日1回朝食後,
201,2,1,,2,610463120,トランサミン錠250mg,2,1,錠
101,3,1,,5
111,3,1,,1日1回朝食後,
201,3,1,,2,612220036,アスベリン錠20 20mg,3,1,錠
101,4,1,,5
111,4,1,,頭痛時,
201,4,1,,2,620002023,カロナール錠200 200mg,2,1,錠



/home/yukim/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: 【般】メキタジン錠3mgwas not in DB and was changed to メキタジン錠3mg
/home/yukim/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: トランサミン錠250mg	was not in DB and was changed to トランサミン錠250mg
/home/yukim/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: アスベリン錠20 20mg 	was not in DB and was changed to アスベリン錠20 20mg


In [16]:
print(create_JAHIS_str(input_list_PHC_with_error,input_type=list))

JAHIS7
1,1,0411306,12,テスト医療機関
5,,,区分保険者扶養者...医師氏名dummy?
11,,テスト患者,テストカンジャ
12,1
13,3440906
22,-1
23,1234,5678,1,
51,4261201
101,1,内服,,分3毎食後服用
111,1,1,,14,
201,1,1,,7,2233002F1ZZZ,カルボシステイン錠250mg,-1,1,ダミー単位
201,1,2,,7,2239001F1ZZZ,アンブロキソール塩酸塩錠15mg,3,1,錠
201,1,3,,2,620379601,フスコデ配合錠,-1,1,錠
201,1,4,,7,6132016F2ZZZ,セフカペンピボキシル塩酸塩錠100mg,-1,1,ダミー単位
101,2,内服,,分2朝・夕食後服用
111,2,1,,28,
201,2,1,,7,2171014G4ZZZ,ニフェジピン徐放錠20mg(24時間持続),2,1,錠
101,3,外用,,1日1回貼付
111,3,1,,ダミー用法,
201,3,1,,7,2649729S3ZZZ,ケトプロフェンテープ40mg(10×14cm非温感),-1,1,ダミー単位

